In [4]:
import sys
sys.path.append('..')
import chessvision.label as label
import chess

In [6]:
from itertools import groupby
def count(groups): 
    for key, group in groups: 
        yield key, sum(1 for _ in group)

## Let's benchmark some implementations of `to_fen`

In [9]:
def to_fen_a(board):
    fen = ""
    empty_count = 0
    for i in range(8):
        for j in range(8):
            piece = board[i*8+j]
            if piece is None:
                empty_count += 1
            else:
                if empty_count:
                    fen += str(empty_count)
                empty_count = 0
                fen += label.str_from_id[piece]
        if empty_count:
            fen += str(empty_count)
            empty_count = 0
        fen += "/"
    return fen

In [10]:
def to_fen_b(board):
    def create_rank(rank):
        r = list()
        for piece, group in groupby(rank):
            if piece is None:
                r.append(str(sum(1 for _ in group)))
            else:
                r.extend(label.str_from_id[piece] for _ in group)
        return ''.join(r)
    return '/'.join(create_rank(board[r:r+8]) for r in range(0, 64, 8))

In [11]:
def to_fen_c(board):
    def create_rank(rank):
        r = ""
        for piece, n in count(groupby(rank)):
            r += str(n) if piece is None else label.str_from_id[piece] * n
        return r
    return '/'.join(create_rank(board[r:r+8]) for r in range(0, 64, 8))

In [12]:
def to_fen_d(board):
    def create_rank(rank):
        return ''.join(str(n) if piece is None else label.str_from_id[piece] * n 
            for piece, n in count(groupby(rank)))
    return '/'.join(create_rank(board[r:r+8]) for r in range(0, 64, 8))

In [13]:
sparse_board = [None for i in range(64)]
%timeit to_fen_a(sparse_board)
%timeit to_fen_b(sparse_board)
%timeit to_fen_c(sparse_board)
%timeit to_fen_d(sparse_board)

8.81 µs ± 195 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
11.8 µs ± 535 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
12.3 µs ± 748 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
14.3 µs ± 275 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
dense_board = [7 for i in range(64)]
%timeit to_fen_a(dense_board)
%timeit to_fen_b(dense_board)
%timeit to_fen_c(dense_board)
%timeit to_fen_d(dense_board)

13.5 µs ± 655 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
15.1 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
11.2 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
14.1 µs ± 461 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
